# Download Landsat 7-9 image subsets over an AOI using AWS

_Last modified 2023-11-1._

This script is run to download Landsat images over the glaciers available through the Amazon Web Services (AWS) s3 bucket. The workflow is streamlined to analyze images for 10s to 100s of glaciers, specifically, the marine-terminating glaciers along the periphery of Greenland. Sections of code that may need to be modified are indicated as below:

    ##########################################################################################

    code to modify

    ##########################################################################################

 
### First, configure your AWS profile to access the Landsat images on the s3 bucket:

Follow instructions at https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html to get required __aws command line software__.

Set up your AWS profile with a payment account. Then configure it to your machine following these steps:

    aws configure --profile terminusmapping
    
Enter in your credentials, which will be stored locally on your computer. The Boto3 package will be used to access your credentials without leaking your keys. **Protect your AWS access keys!** DO NOT print anything that involves your ACCESS_KEY and SECRET_KEY. GitGuardian may help track any leaked keys. In order to use these credentials to download subsets of images from AWS using vsi3, **GDAL version 3.2. or newer must be installed**.

In [1]:
# AWS settings
from rasterio.session import AWSSession
import boto3
import boto3.session

cred = boto3.Session(profile_name='terminusmapping').get_credentials()
ACCESS_KEY = cred.access_key
SECRET_KEY = cred.secret_key
SESSION_TOKEN = cred.token  ## optional


s3client = boto3.client('s3', 
                        aws_access_key_id = ACCESS_KEY, 
                        aws_secret_access_key = SECRET_KEY, 
#                         aws_session_token = SESSION_TOKEN
                       )

######################################################################################
# path to the collection on AWS usgs-landsat s3 bucket:
collectionpath = 'collection02/level-1/standard/' # collection 2 level 1 data being used (contains B8)
######################################################################################

# 1) Set-up: import packages and set paths


In [2]:
import numpy as np
import pandas as pd
import scipy
import math
import subprocess
import os
import shutil
import datetime
import cv2
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob

# geospatial packages
import fiona
import geopandas as gpd
from shapely.geometry import Polygon, Point, LineString
import shapely
from matplotlib.pyplot import imshow
import rasterio as rio

# Enable fiona KML file reading driver
fiona.drvsupport.supported_drivers['KML'] = 'rw'

# distance function
def distance(x1, y1, x2, y2):
    dist = (((x2-x1)**2)+((y2-y1)**2))**(1/2)
    return dist

/Users/jukesliu/opt/anaconda3/envs/autoterm_env/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


### Define paths, satellites, geographic projections:

In [31]:
######################################################################################
# ADJUST THESE VARIABLES:
basepath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/' # folder containing the all glacier shapefile(s)
downloadpath ='/Users/jukesliu/Documents/PECLET/B3/LS_B3/' # folder to eventually contain downloaded Landsat images

sats = ['L8','L9'] # names of landsats to download images from ('L7' for Landsat 7 or 'L8' or both)
# sats = ['L7']
L9_yrs = np.arange(2013,2023).astype(str) 
L8_yrs = np.arange(2013,2023).astype(str) # set target years for L8: 2013-2021
# L7_yrs = np.arange(2012,2013).astype(str) # set target years for L7: 1999-2003
L8_bands = [8] # panchromatic band for L8
L7_bands = [8] # panchromatic band for L7
L9_bands = [8]
######################################################################################

In [32]:
## Read in AOI shapefile and determine source_srs ####################################
AOIpath = '/Users/jukesliu/Documents/PECLET/B3/B3_Box_WGS.shp'
# this path is also where the UTM reprojections of the AOI shp will be stored
AOI = fiona.open(AOIpath).next()
aoi = AOI['geometry']['coordinates'][0]
print(aoi)

source_srs = '4326' # EPSG code for the current projection of the glacier shapefile(s)

# Reproject to WGS if in a different srs
# boxespath = basepath+"Box"+BoxID+"/Box"+BoxID # access the BoxID folders created 
# # construct the gdal command
# rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:"+source_srs+" "
# rp +=boxespath+"_WGS.shp "+boxespath+".shp"
# print("Command:", rp) # check command
# subprocess.run(rp, shell=True, check=True) # run the command on terminal

# # if an error is produced, check the error output on the terminal window that runs this notebook
######################################################################################

[(23.084716348180375, 79.581786551003), (23.995162309702508, 79.79519672199241), (26.644550818095507, 79.40604670929159), (25.715272146199567, 79.20022959626894), (23.084716348180375, 79.581786551003)]


/var/folders/mr/7f69f7y50zj85n420pyvh8cr0000gq/T/ipykernel_18848/3586301238.py:4: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  AOI = fiona.open(AOIpath).next()



# 2) Find all the Landsat footprints that overlap the glaciers

This step requires the **WRS-2_bound_world_0.kml** file containing the footprints of all the Landsat scene boundaries available through the USGS (https://www.usgs.gov/media/files/landsat-wrs-2-scene-boundaries-kml-file). Place this file in your base directory (basepath). 

To check if they overlap the glacier terminus box shapefiles, the box shapefiles must be in WGS84 coordinates (ESPG: 4326). If they are not yet, we use the following GDAL command to reproject them into WGS84:

        ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:NEW_EPSG_NUMBER -s_srs EPSG:OLD_EPSG_NUMBER out.shp in.shp

In [27]:
######################################################################################
# open the kml file with the Landsat path, row footprints:
WRS = fiona.open(basepath+'WRS-2_bound_world_0.kml', driver='KML') # check the path to the world bounds file
print('Landsat footprint file opened.')
######################################################################################

Landsat footprint file opened.


In [28]:
# Grab the WGS84 coordinates of the AOI
points = [] # to hold the box vertices
# read coordinates and convert to a shapely object
for coord_pair in list(aoi): 
    lat = coord_pair[0]; lon = coord_pair[1]        
    point = shapely.geometry.Point(lat, lon) # create shapely point 
    points.append(point) # append to points list
print("coordinates recorded.") # keep track of progress

coordinates recorded.


In [29]:
paths = []; rows = []; boxes = [] # create lists to hold the paths and rows and BoxIDs

#loop through all Landsat scenes (path, row footprints)
for feature in WRS:
    # create shapely polygons from the Landsat footprints
    coordinates = feature['geometry']['coordinates'][0]
    coords = [xy[0:2] for xy in coordinates]
    pathrow_poly = Polygon(coords)
    
    # grab the path and row name from the WRS kml file:
    pathrowname = feature['properties']['Name']  
    path = pathrowname.split('_')[0]; row = pathrowname.split('_')[1]
#     print(path, row)
      
    box_points_in = 0 # counter for number of box_points in the pathrow_geom:
    for i in range(0, len(points)):
        point = points[i]
        if point.within(pathrow_poly): # if the pathrow shape contains the point
            box_points_in = box_points_in+1 # append the counter
    if box_points_in == 5: # if all box vertices are inside the footprint, save the path, row, BoxID
        paths.append('%03d' % int(path))
        rows.append('%03d' % int(row))

# Store in dataframe
boxes_pr_df = pd.DataFrame(list(zip(paths, rows)), columns=['Path', 'Row'])
boxes_pr_df # display

,Path,Row
0,212,003
1,211,003
2,210,003
3,216,002
4,215,002
5,214,002


# 3) Download metadata files from AWS s3 for overlapping Landsat scenes
     
The syntax for listing the Collection 2 Landsat image files AWS s3 bucket is as follows:

    aws s3 ls --profile terminusmapping --request-payer requester s3://usgs-landsat/collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_LS2R_pathrow_yyyyMMdd_yyyyMMdd_02_T1/ 
    
__NOTE: Including the --request-payer requester as part of this line indicates that the referenced user will be charged for data download.__

We can use the paths and rows in the dataframe to access the full Landsat scene list and the corresponding metdata files. Read https://docs.opendata.aws/landsat-pds/readme.html to learn more.
    
The metadata files will be downloaded into folders corresponding to the Landsat footprint, identified by the Path Row numbers:
    
    aws s3api get-object --bucket usgs-landsat --key collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt  --request-payer requester LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt

In [33]:
# Loop through the dataframe containing overlapping path, row info:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; folder_name = 'Path'+p+'_Row'+r+'_c2' # folder name
    bp_out = downloadpath+folder_name+'/' # output path for the downloaded files
    print("Downloaded metadata files are stored in:",bp_out)
    
    # create Path_Row folders if they don't exist already
    if os.path.exists(bp_out):
        print(folder_name, " exists already, skip directory creation")
    else:
        os.mkdir(bp_out)
        print(folder_name+" directory made")
    
    for sat in sats: # for each satellite
        if sat == 'L8':
            collectionfolder = 'oli-tirs/'; years = L8_yrs; prefix='LC08' # set folder, years, file prefix
        elif sat == 'L7':
            collectionfolder = 'etm/'; years = L7_yrs; prefix='LE07' # set folder, years, file prefix
        elif sat == 'L9':
            collectionfolder = 'oli-tirs/'; years = L8_yrs; prefix='LC09' # set folder, years, file prefix
            
        # loop through years
        for year in years:
            # grab list of images in each year, path, row folder
            find_imgs = 'aws s3 ls --profile terminusmapping --request-payer requester s3://usgs-landsat/'
            find_imgs += collectionpath+collectionfolder
            find_imgs += year+'/'+p+'/'+r+'/'

            if subprocess.run(find_imgs,shell=True).returncode != 0:
                print('No results found for '+collectionpath+collectionfolder+year+'/'+p+'/'+r+'/')
                results = [] # empty results
            else:
                result = subprocess.check_output(find_imgs,shell=True) # grab the avilable images
                results = result.split() # split string
            
            imagenames = []
            for line in results: # loop through strings
                line = str(line)
                if prefix in line and 'T1' in line: # find just the Tier-1 images
                    imgname = line[2:-2]; imagenames.append(imgname)

            # download the metadata (MTL.txt) file if it doesn't exist
            for imgname in imagenames:
                if not os.path.exists(bp_out+imgname+'_MTL.txt'): # check in output directory
                    command = 'aws s3api get-object --bucket usgs-landsat --key '+collectionpath+collectionfolder
                    command += year+'/'+p+'/'+r+'/'
                    command += imgname+'/'+imgname+'_MTL.txt'
                    command += ' --profile terminusmapping --request-payer requester '
                    command += bp_out+imgname+'_MTL.txt'
                    print('Downloading', imgname+'_MTL.txt')
                    subprocess.run(command,shell=True,check=True)
                else:
                    print(imgname+'_MTL.txt exists. Skip.')

Downloaded metadata files are stored in: /Users/jukesliu/Documents/PECLET/B3/LS_B3/Path212_Row003_c2/
Path212_Row003_c2  exists already, skip directory creation
                           PRE LC08_L1GT_212003_20130322_20200912_02_T2/
                           PRE LC08_L1GT_212003_20130425_20200913_02_T2/
2021-02-18 16:05:05        927 catalog.json
No results found for collection02/level-1/standard/oli-tirs/2014/212/003/
No results found for collection02/level-1/standard/oli-tirs/2015/212/003/
No results found for collection02/level-1/standard/oli-tirs/2016/212/003/
No results found for collection02/level-1/standard/oli-tirs/2017/212/003/
                           PRE LC08_L1GT_212003_20180322_20200901_02_T2/
                           PRE LC08_L1GT_212003_20180626_20200831_02_T2/
                           PRE LC08_L1GT_212003_20180829_20200831_02_T2/
                           PRE LC08_L1TP_212003_20180423_20201015_02_T2/
                           PRE LC08_L1TP_212003_20180728_2020

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-09T00:16:03+00:00",
    "ContentLength": 12347,
    "ETag": "\"ef2c6f8471339d544c2a158c396aee49\"",
    "VersionId": "IVq8ENcFdSMn98BU9A6C3UucCVmUOgPj",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-10-16T06:04:29+00:00",
    "ContentLength": 12276,
    "ETag": "\"41a508430e0bebd0c7ef45687ec40baa\"",
    "VersionId": "ak8U2YfOlPpuK8IF8wdy7V.HgMV0_UB8",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-09T01:49:49+00:00",
    "ContentLength": 12348,
    "ETag": "\"174c8194908fca950cc33eac7fb4007e\"",
    "VersionId": "upK.E5XdoJ.oKYoKxiGtwzlSMWW9b1ga",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "L

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-31T02:47:36+00:00",
    "ContentLength": 12277,
    "ETag": "\"f18668b6c75f2d6447e8a34e5a0fc5fd\"",
    "VersionId": "RnApost67NoyKPi.uC_ejCT_AevPUbhA",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-31T02:49:07+00:00",
    "ContentLength": 12277,
    "ETag": "\"5354de2b1be6d388778a7fb65bfe3c45\"",
    "VersionId": "W59UG5ChFoZQeddXAtTww7WaE_aW8IYk",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
                           PRE LC08_L1GT_211003_20190318_20200829_02_T2/
                           PRE LC08_L1GT_211003_20190419_20200829_02_T2/
                           PRE LC08_L1GT_211003_20190505_20200828_02_T2/
                           PRE LC08_L1GT_211003_20190521_20200828_02_T2/
                           PRE LC08_L1GT_211003_20190606_20200828_

{
    "AcceptRanges": "bytes",
    "LastModified": "2022-06-03T08:16:04+00:00",
    "ContentLength": 12365,
    "ETag": "\"5005869e630ddca040c58523835d0e71\"",
    "VersionId": "hKQbH9gb_bVWl0SOtl8Dof6wNGTQ4K6E",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2022-07-08T21:20:15+00:00",
    "ContentLength": 12288,
    "ETag": "\"9a9c8647094435422292aaec5fe12017\"",
    "VersionId": "jUqi.fRyd3LwJK4B2wuDB6FheGtsvxhi",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2022-07-26T09:24:52+00:00",
    "ContentLength": 12286,
    "ETag": "\"b142e60487312738563125f86342bd7a\"",
    "VersionId": "7TySoiBJXxU5ihE2HrXjAjFDmekxh1W6",
    "ContentType": "binary/octet-stream",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "Acc

                           PRE LC08_L1GT_211003_20210323_20210401_02_T2/
                           PRE LC08_L1GT_211003_20210424_20210501_02_T2/
                           PRE LC08_L1GT_211003_20210510_20210518_02_T2/
                           PRE LC08_L1GT_211003_20210611_20210622_02_T2/
                           PRE LC08_L1GT_211003_20210814_20210819_02_T2/
                           PRE LC08_L1TP_211003_20210408_20210416_02_T1/
                           PRE LC08_L1TP_211003_20210526_20210529_02_T2/
                           PRE LC08_L1TP_211003_20210729_20210804_02_T1/
                           PRE LC08_L1TP_211003_20210830_20210909_02_T1/
                           PRE LC08_L1TP_211003_20210915_20210924_02_T1/
2021-09-24 07:56:14       4315 catalog.json
                           PRE LC08_L1GT_211003_20220310_20220315_02_T2/
                           PRE LC08_L1GT_211003_20220427_20220503_02_T2/
                           PRE LC08_L1GT_211003_20220801_20220806_02_T2/
       

...

                           PRE LC08_L1GT_214002_20150328_20200909_02_T2/
                           PRE LC08_L1GT_214002_20150413_20200909_02_T2/
                           PRE LC08_L1GT_214002_20150616_20200909_02_T2/
                           PRE LC08_L1GT_214002_20150718_20200908_02_T2/
                           PRE LC08_L1GT_214002_20150904_20201015_02_T2/
                           PRE LC08_L1TP_214002_20150312_20200910_02_T2/
                           PRE LC08_L1TP_214002_20150429_20200909_02_T2/
                           PRE LC08_L1TP_214002_20150515_20200909_02_T1/
                           PRE LC08_L1TP_214002_20150531_20200909_02_T2/
                           PRE LC08_L1TP_214002_20150702_20200909_02_T1/
                           PRE LC08_L1TP_214002_20150803_20201016_02_T1/
                           PRE LC08_L1TP_214002_20150819_20200908_02_T1/
                           PRE LC08_L1TP_214002_20150920_20200908_02_T1/
2021-02-18 15:55:24       3776 catalog.json
{
    "

                           PRE LC08_L1GT_214002_20180523_20200901_02_T2/
                           PRE LC08_L1GT_214002_20180608_20200831_02_T2/
                           PRE LC08_L1GT_214002_20180624_20200831_02_T2/
                           PRE LC08_L1TP_214002_20180320_20200901_02_T1/
                           PRE LC08_L1TP_214002_20180405_20200901_02_T2/
                           PRE LC08_L1TP_214002_20180421_20201015_02_T2/
                           PRE LC08_L1TP_214002_20180507_20200901_02_T2/
                           PRE LC08_L1TP_214002_20180710_20200831_02_T1/
                           PRE LC08_L1TP_214002_20180726_20200831_02_T2/
                           PRE LC08_L1TP_214002_20180811_20200831_02_T1/
                           PRE LC08_L1TP_214002_20180827_20200831_02_T1/
                           PRE LC08_L1TP_214002_20180912_20200830_02_T1/
                           PRE LC08_L1TP_214002_20180928_20200830_02_T1/
2021-02-18 15:41:35       3776 catalog.json
{
    "

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-12-05T06:10:29+00:00",
    "ContentLength": 12364,
    "ETag": "\"7fd789876b0bb314d9a3264a9dcdad6a\"",
    "VersionId": "hcyr2OAGJkxsLyQGxSjO_RqSo0ONMVWV",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-12-04T09:47:21+00:00",
    "ContentLength": 12367,
    "ETag": "\"8bc9378849aa08ff1c3e9e381bf0bc7a\"",
    "VersionId": "kauJTVmtLDx4EgxenPspeV65r2NtPvoO",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
                           PRE LC08_L1GT_214002_20210312_20210317_02_T2/
                           PRE LC08_L1GT_214002_20210328_20210401_02_T2/
                           PRE LC08_L1GT_214002_20210429_20210508_02_T2/
                           PRE LC08_L1GT_214002_20210515_20210525_02_T2/
                           PRE LC08_L1GT_214002_20210616_20210622_

                           PRE LC08_L1GT_214002_20160314_20200907_02_T2/
                           PRE LC08_L1GT_214002_20160330_20200907_02_T2/
                           PRE LC08_L1GT_214002_20160415_20200907_02_T2/
                           PRE LC08_L1GT_214002_20160501_20200907_02_T2/
                           PRE LC08_L1GT_214002_20160517_20200907_02_T2/
                           PRE LC08_L1GT_214002_20160602_20200906_02_T2/
                           PRE LC08_L1GT_214002_20160618_20200906_02_T2/
                           PRE LC08_L1GT_214002_20160704_20200906_02_T2/
                           PRE LC08_L1GT_214002_20160720_20200906_02_T2/
                           PRE LC08_L1GT_214002_20160805_20200906_02_T2/
                           PRE LC08_L1TP_214002_20160821_20200906_02_T1/
                           PRE LC08_L1TP_214002_20160906_20200906_02_T2/
                           PRE LC08_L1TP_214002_20160922_20200906_02_T1/
2021-02-18 15:50:49       3776 catalog.json
       

{
    "AcceptRanges": "bytes",
    "LastModified": "2023-04-02T15:42:54+00:00",
    "ContentLength": 12133,
    "ETag": "\"f77021a869333916e1367ca223b8caeb\"",
    "VersionId": "OOQKj9KFDHNz3Z_O4cenxlOHsYJXJYJp",
    "ContentType": "binary/octet-stream",
    "ServerSideEncryption": "AES256",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
{
    "AcceptRanges": "bytes",
    "LastModified": "2023-03-27T08:42:39+00:00",
    "ContentLength": 12205,
    "ETag": "\"2e66b007ed30dc24f480040454be8720\"",
    "VersionId": "DBqocqqZFL7h.58lBCKYnL5JtlZ9qULQ",
    "ContentType": "binary/octet-stream",
    "ServerSideEncryption": "AES256",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}


# 4) Download QAPIXEL band over terminus box to determine cloud cover

If the terminus box shapefiles were not originally in UTM projection, will need to reproject them into UTM to match the Landsat projection. The code automatically finds the UTM zones from the metadata files and fills in the following syntax to reproject:
    
    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326zone output.shp input.shp
    
#### If the terminus box shapefiles are already in UTM projection, skip the following cell and rename the files to end with "\_UTM\_##.shp" where ## corresponds to the zone number (e.g., "\_UTM\_07.shp", "\_UTM\_21.shp").

In [16]:
# If pyproj is not showing up:
os.environ['PROJ_LIB'] = '/Users/jukesliu/opt/anaconda3/envs/autoterm_env/share/proj'

In [34]:
zones = {} # initialize dictionary to hold UTM zone for each Landsat scene path row
zone_list = [] # list of zones

# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; folder_name = 'Path'+p+'_Row'+r+'_c2' # Landsat path and row
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    
    if len(os.listdir(pr_folderpath)) > 0: # if there are files in the folder
        # grab UTM Zone from the first metadata file
        mtl_scene = glob.glob(pr_folderpath+'*_MTL.txt')[0]
        mtl = open(mtl_scene, 'r')
        
        # loop through lines in the metadata file to find the UTM ZONE
        for line in mtl:  
            variable = line.split("=")[0] # grab the variable name
            if ("UTM_ZONE" in variable):
                zone = '%02d' % int(line.split("=")[1][1:-1]) # grab the 2-digit zone number
                zones.update({folder_name: zone}); zone_list.append(zone) # add to zone lists
                break
                
        # reproject shapefile(s) into UTM
        zone = zones[folder_name]
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" '+AOIpath[:-4]+'_UTM_'+zone+'.shp '+AOIpath
        rp_shp += ' -t_srs EPSG:326'+zone
        subprocess.run(rp_shp, shell=True,check=True)
        
    else: # if no files in folder, zone = nan, must fill in manually
        zone_list.append(np.nan)
        
boxes_pr_df['Zone'] = zone_list # add to the path row dataframe
boxes_pr_df.head()

,Path,Row,Zone
0,212,003,35
1,211,003,35
2,210,003,35
3,216,002,35
4,215,002,35


Use GDAL and __vsi3__ link to download subset of the quality band we will use to determine cloud cover over the terminus:

    gdalwarp -cutline path_to_shp.shp -crop_to_cutline /vsi3/usgs-landsat/collection02/level-1/standard/oli-tirs/yyyy/path/row/scene/scene_QA_PIXEL.TIF path_to_subset_QA_PIXEL.TIF


In [41]:
# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; zone = row['Zone'] # grab path, row, zone
    folder_name = 'Path'+p+'_Row'+r+'_c2'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp_rp = AOIpath[:-4]+'_UTM_'+zone # path to the UTM projected box shapefile

    files = os.listdir(pr_folderpath) # grab the names of the Landsat scenes
    
    # for all files in the path row folders
    for file in files:
        scene = file[:40] # slice the filename to grab the scene name

        if scene.startswith('L') and 'T1' in scene: # L1TP scenes
            scene_year = scene[17:21] # grab the year from the scene name
            
            if scene.startswith('LC08') or scene.startswith('LC09'):
                collectionfolder='oli-tirs/'
            elif scene.startswith('LE07'):
                collectionfolder='etm/'
                
            # set path to the QA pixel Landsat files
            pathtoQAPIXEL='/vsis3/usgs-landsat/'+collectionpath+collectionfolder
            pathtoQAPIXEL+=scene_year+'/'
            pathtoQAPIXEL+=p+'/'+r+'/'
            pathtoQAPIXEL+=scene+'/'+scene+"_QA_PIXEL.TIF"
            
            # set path to the subset QA pixel files inside the path row folders
            subsetout = pr_folderpath+scene+'_QA_PIXEL.TIF' 
            
            # if the file hasn't already been downloaded
            if not os.path.exists(subsetout): #and not scene_year.startswith('202'): # EXCLUDE CERTAIN YEARS
                print('Downloading', scene)
                # construct download command
                QAPIXEL_dwnld_cmd='gdalwarp -overwrite -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '
                QAPIXEL_dwnld_cmd+= pathtoQAPIXEL+' '+subsetout
                QAPIXEL_dwnld_cmd+=' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2'
                QAPIXEL_dwnld_cmd+=' --config AWS_SECRET_ACCESS_KEY '+SECRET_KEY
                QAPIXEL_dwnld_cmd+=' --config AWS_ACCESS_KEY_ID '+ACCESS_KEY

                subprocess.run(QAPIXEL_dwnld_cmd, shell=True, check=True)

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2013/210/003/LC08_L1TP_210003_20130902_20200912_02_T1/LC08_L1TP_210003_20130902_20200912_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2014/210/003/LC08_L1TP_210003_20140905_20200911_02_T1/LC08_L1TP_210003_20140905_20200911_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/210/003/LC08_L1TP_210003_20150706_20200909_02_T1/LC08_L1TP_210003_20150706_20200909_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/210/003/

Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/210/003/LC08_L1TP_210003_20170711_20201015_02_T1/LC08_L1TP_210003_20170711_20201015_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2014/210/003/LC08_L1TP_210003_20140820_20200911_02_T1/LC08_L1TP_210003_20140820_20200911_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/210/003/LC08_L1TP_210003_20180730_20200831_02_T1/LC08_L1TP_210003_20180730_20200831_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/210/003/LC08_L1TP_210003_20220826_20220909_02_T1/LC0

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2016/216/002/LC08_L1TP_216002_20160718_20200906_02_T1/LC08_L1TP_216002_20160718_20200906_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/216/002/LC08_L1TP_216002_20210630_20210708_02_T1/LC08_L1TP_216002_20210630_20210708_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2013/216/002/LC08_L1TP_216002_20130421_20200912_02_T1/LC08_L1TP_216002_20130421_20200912_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/216/002/

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/216/002/LC08_L1TP_216002_20170619_20200903_02_T1/LC08_L1TP_216002_20170619_20200903_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/216/002/LC08_L1TP_216002_20180403_20200901_02_T1/LC08_L1TP_216002_20180403_20200901_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/216/002/LC08_L1TP_216002_20200830_20200906_02_T1/LC08_L1TP_216002_20200830_20200906_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/216/002/

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/215/002/LC08_L1TP_215002_20200722_20200911_02_T1/LC08_L1TP_215002_20200722_20200911_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/215/002/LC09_L1TP_215002_20220704_20230408_02_T1/LC09_L1TP_215002_20220704_20230408_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/215/002/LC08_L1TP_215002_20210810_20210819_02_T1/LC08_L1TP_215002_20210810_20210819_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/215/002/

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/215/002/LC09_L1TP_215002_20220720_20230406_02_T1/LC09_L1TP_215002_20220720_20230406_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/215/002/LC08_L1TP_215002_20190314_20200829_02_T1/LC08_L1TP_215002_20190314_20200829_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/215/002/LC08_L1TP_215002_20150319_20200909_02_T1/LC08_L1TP_215002_20150319_20200909_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/215/002/

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/214/002/LC08_L1TP_214002_20200901_20200906_02_T1/LC08_L1TP_214002_20200901_20200906_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/214/002/LC08_L1TP_214002_20210803_20210811_02_T1/LC08_L1TP_214002_20210803_20210811_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC09_L1TP_214002_20220526_20230415_02_T1/LC09_L1TP_214002_20220526_20230415_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2016/214/002/

Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/214/002/LC08_L1TP_214002_20170605_20200903_02_T1/LC08_L1TP_214002_20170605_20200903_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/214/002/LC08_L1TP_214002_20200613_20200824_02_T1/LC08_L1TP_214002_20200613_20200824_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220603_20220610_02_T1/LC08_L1TP_214002_20220603_20220610_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 2388P x 2254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/214/002/

# 5) Download non-cloudy Landsat images from AWS

To remove cloudy images, we will find the number of pixels in our terminus box that exceed a threshold value in the QA_PIXEL band corresponding to cloud and cloud shadow likelihood. If the fraction of cloudy pixels with values is above the threshold, we won't download the image. See Landsat Collection 2 Level 2 Science Product Guide for [Landsat 8](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat-8-9-C2-L2-ScienceProductGuide-v4.pdf) and [Landsat 7](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-1618_Landsat-4-7_C2-L2-ScienceProductGuide-v3.pdf) more information on how the QA_PIXEL threshold values are chosen.

Additionally, we remove images that are primarily black (fill value of 1 in QA_PIXEL band). This ensures that the scenes that cut off halfway across the glacier are not included in further analysis. The fill percent threshold may need to be adjusted.

In [42]:
######################################################################################
# These are the recommended values based on the Collection 2 Level 2 Science Product Guide.
# Adjust thresholds here:
QAPIXEL_thresh_lower_L7 = 5696.0 # minimum QA pixel value threshold to be considered cloud for L7 images
QAPIXEL_thresh_upper_L7 = 7568.0 # maximum QA pixel value threshold to be considered cloud for L7 images

# Landsat 8 requires two lower thresholds, we selct between 22280 and 24472 and above 54596
QAPIXEL_thresh_lower_L8 = 22080.0 # minimum QA pixel value threshold to be considered cloud for L8 images
QAPIXEL_thresh_upper_L8 = 30048.0 # maximum QA pixel value threshold to be considered cloud for L8 images
QAPIXEL_thresh2_lower_L8 = 54596.0 # 2nd minimum QA pixel value threshold to be considered cloud for L8 images

cpercent_thresh = 100.0 # maximum cloud cover % in terminus box
fpercent_thresh = 60.0 # maximum fill % in terminus box
######################################################################################

In [45]:
# Download images that pass these thresholds:
for index, row in boxes_pr_df.iterrows():
    # grab paths
    p = row['Path']; zone = row['Zone']; r = row['Row']
    folder_name = 'Path'+p+'_Row'+r+'_c2'
    pr_folderpath = downloadpath+folder_name+'/'
    bp_out = downloadpath
    
    print(pr_folderpath)
    
    # path to the shapefile covering the region that will be downloaded
    pathtobuffer = AOIpath[:-4]+'_UTM_'+zone+'.shp' # just the box
    
    for scene in os.listdir(pr_folderpath):
        if scene.startswith('L') and scene.endswith(".TIF") and 'T1' in scene and 'L1TP' in scene: # For Tier-1 images
            scene = scene[:40] # scene name
            year = scene[17:21] # grab acquisition year
            
            QApixelpath = pr_folderpath+scene+'_QA_PIXEL.TIF' # path to QA_PIXEL file
            subsetQApixel = mpimg.imread(QApixelpath) # read in QAPIXEL file as numpy array
            totalpixels = subsetQApixel.shape[0]*subsetQApixel.shape[1] # count total number of pixels
            
            if scene.startswith("LC08") or scene.startswith('LC09'): # Landsat 8 or Landsat 9
                collectionfolder = 'oli-tirs/'; bands = L8_bands; 
                # countcloudy pixels based on thresholds:
                cloudQApixel = subsetQApixel[((subsetQApixel >= QAPIXEL_thresh_lower_L8) &
                                             (subsetQApixel < QAPIXEL_thresh_upper_L8) | 
                                             (subsetQApixel >= QAPIXEL_thresh2_lower_L8))]
                
            elif scene.startswith("LE07"): # Landsat 7
                collectionfolder = 'etm/'; bands = L7_bands
                # countcloudy pixels based on thresholds:
                cloudQApixel = subsetQApixel[((subsetQApixel >= QAPIXEL_thresh_lower_L7) & 
                                             (subsetQApixel < QAPIXEL_thresh_upper_L7))]
 
            # calculate percentages of cloud and fill pixels
            fillQApixel = subsetQApixel[subsetQApixel < 2.0] # fill pixels (value = 0 or 1)
            cloudpixels = len(cloudQApixel); fillpixels = len(fillQApixel) # count the cloudy and fill pixels
            cloudpercent = int(float(cloudpixels)/float(totalpixels)*100) # calculate percent cloudy
            fillpercent = int(float(fillpixels)/float(totalpixels)*100) # calculate percent fill
            
            # evaluate thresholds
            if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
                # download the bands for that scene into your scene folders:
                for band in bands:
                        band = str(band) # string format
                        
                        # input path to your bands in AWS:
                        pathin = '/vsis3/usgs-landsat/'+collectionpath+collectionfolder+year+'/'+p+"/"+r+"/"+scene+"/"+scene+"_B"+band+".TIF"
                        
                        outfilename = scene+"_B"+band+'.TIF' # output file name
                        pathout = downloadpath+outfilename # full output file path
                        
                        # if the file hasn't already been downloaded
                        if not os.path.exists(pathout):
                            # download
                            download_cmd = 'gdalwarp -overwrite -cutline '+pathtobuffer+' -crop_to_cutline '+pathin+' '+pathout
                            download_cmd+=' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2'
                            download_cmd+=' --config AWS_SECRET_ACCESS_KEY '+SECRET_KEY
                            download_cmd+=' --config AWS_ACCESS_KEY_ID '+ACCESS_KEY   
                            print('Downloading:', outfilename)
                            subprocess.run(download_cmd, shell=True, check=True)
                        else:
                            print(outfilename, 'exists')
            else:
                print(scene, 'failed cloud & fill thresholds: Cloud % ', cloudpercent, 'Fill %', fillpercent)

/Users/jukesliu/Documents/PECLET/B3/LS_B3/Path212_Row003_c2/
LC08_L1TP_212003_20190917_20200826_02_T1_B8.TIF exists
LC08_L1TP_212003_20190715_20200827_02_T1_B8.TIF exists
LC08_L1TP_212003_20180728_20200831_02_T1_B8.TIF exists
LC08_L1TP_212003_20200428_20200820_02_T1_B8.TIF exists
/Users/jukesliu/Documents/PECLET/B3/LS_B3/Path211_Row003_c2/
LC08_L1TP_211003_20210830_20210909_02_T1_B8.TIF exists
LC08_L1TP_211003_20160613_20200906_02_T1_B8.TIF exists
LC08_L1TP_211003_20150713_20200908_02_T1_B8.TIF exists
LC08_L1TP_211003_20200827_20200906_02_T1_B8.TIF exists
LC08_L1TP_211003_20210408_20210416_02_T1_B8.TIF exists
LC08_L1TP_211003_20160731_20200906_02_T1_B8.TIF exists
LC08_L1TP_211003_20180907_20200831_02_T1_B8.TIF exists
LC08_L1TP_211003_20220902_20220910_02_T1_B8.TIF exists
LC08_L1TP_211003_20170803_20200903_02_T1_B8.TIF exists
LC08_L1TP_211003_20180806_20200831_02_T1_B8.TIF exists
LC08_L1TP_211003_20220529_20220603_02_T1_B8.TIF exists
LC08_L1TP_211003_20180721_20200831_02_T1_B8.TIF exist

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20200729_20200908_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/216/002/LC08_L1TP_216002_20200729_20200908_02_T1/LC08_L1TP_216002_20200729_20200908_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/216/002/LC08_L1TP_216002_20200729_20200908_02_T1/LC08_L1TP_216002_20200729_20200908_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/216/002/LC08_L1TP_216002_20200729_20200908_02_T1/LC08_L1TP_216002_20200729_20200908_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC08_L1TP_216002_20200729_20200908_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20160819_20200906_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20140915_20200910_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2014/216/002/LC08_L1TP_216002_20140915_20200910_02_T1/LC08_L1TP_216002_20140915_20200910_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2014/216/002/LC08_L1TP_216002_20140915_20200910_02_T1/LC08_L1TP_216002_20140915_20200910_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2014/216/002/LC08_L1TP_216002_20140915_20200910_02_T1/LC08_L1TP_216002_20140915_20200910_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC08_L1TP_216002_20140915_20200910_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20150918_20200908_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC09_L1TP_216002_20220812_20230402_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/216/002/LC09_L1TP_216002_20220812_20230402_02_T1/LC09_L1TP_216002_20220812_20230402_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/216/002/LC09_L1TP_216002_20220812_20230402_02_T1/LC09_L1TP_216002_20220812_20230402_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/216/002/LC09_L1TP_216002_20220812_20230402_02_T1/LC09_L1TP_216002_20220812_20230402_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC09_L1TP_216002_20220812_20230402_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20210918_20210924_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20180622_20200831_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/216/002/LC08_L1TP_216002_20180622_20200831_02_T1/LC08_L1TP_216002_20180622_20200831_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/216/002/LC08_L1TP_216002_20180622_20200831_02_T1/LC08_L1TP_216002_20180622_20200831_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/216/002/LC08_L1TP_216002_20180622_20200831_02_T1/LC08_L1TP_216002_20180622_20200831_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC08_L1TP_216002_20180622_20200831_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_216002_20160702_20200906_02_T1_B8.TIF
Creating output file t

...

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20220603_20220610_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220603_20220610_02_T1/LC08_L1TP_214002_20220603_20220610_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220603_20220610_02_T1/LC08_L1TP_214002_20220603_20220610_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220603_20220610_02_T1/LC08_L1TP_214002_20220603_20220610_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC08_L1TP_214002_20220603_20220610_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20160922_20200906_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20180912_20200830_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/214/002/LC08_L1TP_214002_20180912_20200830_02_T1/LC08_L1TP_214002_20180912_20200830_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/214/002/LC08_L1TP_214002_20180912_20200830_02_T1/LC08_L1TP_214002_20180912_20200830_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/214/002/LC08_L1TP_214002_20180912_20200830_02_T1/LC08_L1TP_214002_20180912_20200830_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC08_L1TP_214002_20180912_20200830_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20190408_20200829_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC09_L1TP_214002_20220729_20230405_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC09_L1TP_214002_20220729_20230405_02_T1/LC09_L1TP_214002_20220729_20230405_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC09_L1TP_214002_20220729_20230405_02_T1/LC09_L1TP_214002_20220729_20230405_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC09_L1TP_214002_20220729_20230405_02_T1/LC09_L1TP_214002_20220729_20230405_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC09_L1TP_214002_20220729_20230405_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20130914_20200913_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20220822_20220923_02_T1_B8.TIF
Creating output file that is 4778P x 4509L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220822_20220923_02_T1/LC08_L1TP_214002_20220822_20220923_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220822_20220923_02_T1/LC08_L1TP_214002_20220822_20220923_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/214/002/LC08_L1TP_214002_20220822_20220923_02_T1/LC08_L1TP_214002_20220822_20220923_02_T1_B8.TIF to destination /Users/jukesliu/Documents/PECLET/B3/LS_B3/LC08_L1TP_214002_20220822_20220923_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_214002_20140715_20200911_02_T1_B8.TIF
Creating output file t

...10...20...30...40...50...60...70...80...90...100 - done.


# 6) Delete all quality band files (*QA_PIXEL.TIF) to save space

These files will not be needed after the download step, so they can be removed to save space.

In [46]:
# Grab all path row folder names from boxes_pr_df:
paths = boxes_pr_df['Path']; rows = boxes_pr_df['Row']
for a in range(0, len(paths)): # look in each path row folder
    folder_name = 'Path'+paths[a]+'_Row'+rows[a]+'_c2'
    folderpath = downloadpath+folder_name+'/'

    # remove all files with QA_PIXEL in the name
    for file in os.listdir(folderpath):
        if 'QA_PIXEL' in file:
            os.remove(folderpath+file)